# Clase Tutorial 7: Regresión Lineal Localmente Ponderada

## Ajustando los datos

- Supongamos que tenemos el conjunto de datos de precios de casas, que muestra cómo el precio de una casa varía con su tamaño:

<div style="text-align: center;">
    <img style="width: 50%;" src="img/house_prices.jpg">
</div>

- Una pregunta que debemos hacernos cuando buscamos ajustar un modelo a los datos es: _¿Cuáles son los features que necesito/quiero?_
- Podriamos ajustar usando...
    - Una recta de la forma: $w_0 + w_1 x_1$
    - Una cuadrática de la forma: $w_0 + w_1 x_1 + w_2 x_2^2 $
    - Un ajuste personalizado como: $w_0 + w_1 x + w_2 \sqrt x + w_3 log(x) $
        - En este caso, deberiamos definir nuestras features como: 
            - $x_1 = x$
            - $x_2 = \sqrt x$
            - $x_3 = log(x)$

- Al definir estas nuevas características, la maquinaria de regresión lineal se presta naturalmente para ajustarse a estas funciones de características de entrada en su conjunto de datos.

## Algoritmos de selección de características
- Son algoritmos para decidir de forma automática cual es el conjunto de features que mejor ajusta a nuestros datos.
- Permiten responder la pregunta: _¿Cuál es la combinación de __funciones de nuestras features de entrada__ que resultan en la mejor performance del modelo?_
    - En otras palabras, ¿qué funciones de x (por ejemplo, $x^2$, $\sqrt x$, $log(x)$, $x^3$, $x^{2/3}$) resultan como features apropiadas para usar? 
- Para ajustar a un dataset que es inherentemente no lineal, una línea recta no sería la mejor opción y tendríamos que explorar polinomios.
- Para abordar el problema de ajustar datos que no son lineales y descubrir la mejor combinación de funciones de sus características a utilizar, entra en juego la regresión lineal localmente ponderada.

## Regresión Lineal Localmente Ponderada
- Es un algoritmo que modifica la regresión lineal para adaptarla a funciones no lineales.
- Consideremos el problema de predecir $y$ a partir de los valores de $x$. 

#### 1️⃣ Si ajustamos $y = w_0 + w_1 x$:

> ➡️ Vemos que los datos no están realmente en línea recta, por lo que el ajuste no es muy bueno. __Underfitting__
<div style="text-align: center;">
    <img style="width: 50%;" src="img/lineal1.jpg">
</div>



#### 2️⃣ Si ajustamos:  $y = w_0 + w_1 x + w_2 x^2$ (agregamos una feature más $x^2$):

> ➡️ Vemos que obtenemos un mejor ajuste a los datos. __Good fit__
<div style="text-align: center;">
    <img style="width: 50%;" src="img/lineal2.jpg">
</div>

- Podría parecer que cuantas más features agregamos, mejor. Pero ya vimos que hay cierto riesgo en agregar demasiadas features.

#### 3️⃣ Si ajustamos $y = w_0 + w_1 x + w_2 x^2 + w_3 x^3 + w_4 x^4 + w_5 x^5$ (polinomio de orden 5):

> ➡️ Vemos que aunque la curva ajustada pasa perfectamente los datos, no esperamos que sea un buen predictor de precios de casas. __Overfitting__

<div style="text-align: center;">
    <img style="width: 50%;" src="img/lineal3.jpg">
</div>
